In [ ]:
import os
import torch
import myServices 
import tailsImportAndMosaic
import whiteboxApplications

In [ ]:
# Sets the device for pytorch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')